Author: Emily Wong \
Last Updated: August 29, 2024

# 1. Import libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Data wrangling
import pandas as pd
import numpy as np

# Stats
import statsmodels
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats.contingency import odds_ratio
import scipy
from scipy.stats import ttest_ind, ttest_1samp

# Visualisation
import graphviz
import matplotlib.pyplot as plt
import seaborn as sns

# 2. Import Data

In [3]:
all_data = pd.read_excel("De-identified PMAD data.xlsx")
print('The full dataset consists of',len(all_data),'electronic health records.')

The full dataset consists of 19790 electronic health records.


In [4]:
# Predictor variables
var = ['MOM_PAT_ID','MOM_AGE','MOM_RACE','ETHNIC_GROUP','MARITAL_STATUS','FINANCIAL_CLASS',
       'LBW','PTB',
       'DELIVERY_METHOD','NICU_ADMIT','MFCU_ADMIT',
       'PREE','GDM','GHTN',
       'MOM_BMI','MOM_LOS','CHILD_LOS',
       'HIST_ANXIETY','HIST_DEPRESS','HIST_BIPOLAR','HIST_PMAD','MENTAL_HEALTH_DX_CUTOFF',
       'MED_PSYCH','MED_CARDIO']

# 3. Demographics

In [6]:
cols = var.copy()
cols.append('PMAD_risk') # PMAD_risk is the higher of the two risk ratings (PHQ-9, EPDS) if a patient was administered both.
demo = all_data[cols]

# keep only complete data
demo = demo.dropna()

## 3.1. Age

In [8]:
print("Min Age:",min(demo['MOM_AGE']))
print("Max Age:",max(demo['MOM_AGE']))
print("Mean Age:",np.round(np.mean(demo['MOM_AGE']),2))
print("SD Age:",np.round(np.std(demo['MOM_AGE']),2))

Min Age: 14.0
Max Age: 59.0
Mean Age: 34.13
SD Age: 4.86


## 3.2. Race and Ethnicity

In [10]:
print("------------RACE/ETHNIC COUNTS------------")
race = demo['MOM_RACE']
ethnic = demo['ETHNIC_GROUP']
print(pd.DataFrame(pd.crosstab(index=race, columns=ethnic)))

------------RACE/ETHNIC COUNTS------------
ETHNIC_GROUP                                        Hispanic  Non-Hispanic  \
MOM_RACE                                                                     
Asian or Native Hawaiian or Other Pacific Islander        21          2331   
Black or African American                                 53          1346   
Hispanic White                                          1842             0   
Multiracial                                              140           420   
Other                                                   1000          1113   
Unknown                                                   33            40   
White                                                      0         10856   

ETHNIC_GROUP                                        Unknown  
MOM_RACE                                                     
Asian or Native Hawaiian or Other Pacific Islander       19  
Black or African American                                 3  
Hispan

In [11]:
perc_white = np.round(100*10855/len(demo),0)
print(perc_white,'% identified as non-Hispanic White.')

56.0 % identified as non-Hispanic White.


In [12]:
demo.groupby('MOM_RACE')['MOM_RACE'].count()

MOM_RACE
Asian or Native Hawaiian or Other Pacific Islander     2371
Black or African American                              1402
Hispanic White                                         1842
Multiracial                                             606
Other                                                  2146
Unknown                                                 121
White                                                 10942
Name: MOM_RACE, dtype: int64

In [13]:
100*demo.groupby('MOM_RACE')['MOM_RACE'].count()/len(demo['MOM_RACE'])

MOM_RACE
Asian or Native Hawaiian or Other Pacific Islander    12.202779
Black or African American                              7.215646
Hispanic White                                         9.480185
Multiracial                                            3.118888
Other                                                 11.044776
Unknown                                                0.622748
White                                                 56.314977
Name: MOM_RACE, dtype: float64

In [14]:
cols = var.copy()
cols.append('PHQ9_risk2')
phq9 = all_data[cols]
phq9 = phq9.dropna()            # keep only complete data

In [15]:
cols = var.copy()
cols.append('EPDS_risk2')
epds = all_data[cols]
epds = epds.dropna()            # keep only complete data

In [16]:
print('The final dataset used for model training and evaluation consists of',len(phq9),'PHQ-9 records and',
      len(epds),'EPDS records, with a total of',len(demo),'unique patients.')

The final dataset used for model training and evaluation consists of 11377 PHQ-9 records and 8658 EPDS records, with a total of 19430 unique patients.


In [17]:
phq9_ids = phq9['MOM_PAT_ID']
epds_ids = epds['MOM_PAT_ID']

both_ids = pd.Series(np.intersect1d(phq9_ids,epds_ids))
print('During the transition from the PHQ-9 to the EPDS,',len(both_ids),
      'patients with complete data received both measures and thus were included in both sets of present study analyses.')

During the transition from the PHQ-9 to the EPDS, 605 patients with complete data received both measures and thus were included in both sets of present study analyses.


In [18]:
print("------------PHQ9 PREV------------")
phq9_prev = pd.DataFrame()
phq9_prev = pd.DataFrame(phq9.groupby(['MOM_RACE'])['PHQ9_risk2'].agg(['mean','count']))
phq9_prev

------------PHQ9 PREV------------


,mean,count
MOM_RACE,,
Asian or Native Hawaiian or Other Pacific Islander,0.043696,1396
Black or African American,0.063855,830
Hispanic White,0.064366,1072
Multiracial,0.073333,300
Other,0.040429,1212
Unknown,0.101695,59
White,0.036878,6508


In [19]:
x_white = int(np.sum(phq9['PHQ9_risk2'][phq9['MOM_RACE']=='White']))
n_white = int(len(phq9['PHQ9_risk2'][phq9['MOM_RACE']=='White']))

x_non_white = int(np.sum(phq9['PHQ9_risk2'][phq9['MOM_RACE']!='White']))
n_non_white = int(len(phq9['PHQ9_risk2'][phq9['MOM_RACE']!='White']))

X = np.array([x_white,x_non_white])
N = np.array([n_white,n_non_white])

test_stat, pval = proportions_ztest(count=X, nobs=N, alternative='two-sided')
print('NHW vs Rest',':','Two sided z-test: z = {:.4f}, p value = {:.4f}'.format(test_stat, pval))

NHW vs Rest : Two sided z-test: z = -4.2537, p value = 0.0000


In [20]:
res = odds_ratio([[x_non_white, x_white], [n_non_white-x_non_white, n_white-x_white]])
print('PHQ9 OR:', res.statistic)
res.confidence_interval(confidence_level=0.95)

PHQ9 OR: 1.4732259228681122


ConfidenceInterval(low=1.2261589701695317, high=1.7706428911684142)

In [21]:
print("------------EPDS PREV------------")
epds_prev = pd.DataFrame()
epds_prev = pd.DataFrame(epds.groupby(['MOM_RACE'])['EPDS_risk2'].agg(['mean','count']))
epds_prev

------------EPDS PREV------------


,mean,count
MOM_RACE,,
Asian or Native Hawaiian or Other Pacific Islander,0.166200,1071
Black or African American,0.131045,641
Hispanic White,0.123798,832
Multiracial,0.108504,341
Other,0.109671,1003
Unknown,0.194030,67
White,0.100149,4703


In [22]:
x_white = int(np.sum(epds['EPDS_risk2'][epds['MOM_RACE']=='White']))
n_white = int(len(epds['EPDS_risk2'][epds['MOM_RACE']=='White']))

x_non_white = int(np.sum(epds['EPDS_risk2'][epds['MOM_RACE']!='White']))
n_non_white = int(len(epds['EPDS_risk2'][epds['MOM_RACE']!='White']))

X = np.array([x_white,x_non_white])
N = np.array([n_white,n_non_white])

test_stat, pval = proportions_ztest(count=X, nobs=N, alternative='two-sided')
print('NHW vs Rest',':','Two sided z-test: z = {:.4f}, p value = {:.4f}'.format(test_stat, pval))

NHW vs Rest : Two sided z-test: z = -4.7349, p value = 0.0000


In [23]:
res = odds_ratio([[x_non_white, x_white], [n_non_white-x_non_white, n_white-x_white]])
print('PHQ9 OR:', res.statistic)
res.confidence_interval(confidence_level=0.95)

PHQ9 OR: 1.3752252370882407


ConfidenceInterval(low=1.2022047592244918, high=1.5734647870884093)

In [50]:
# save this file and output as html
import os
os.system('jupyter nbconvert --to html Demographics.ipynb')

[NbConvertApp] Converting notebook Demographics.ipynb to html
[NbConvertApp] Writing 622802 bytes to Demographics.html


0